In [2]:
target_number

3

In [15]:
str(date.today() - timedelta(days=4))

'2023-05-14'

In [16]:
from flask import Flask, render_template, request
from datetime import date, timedelta,datetime
import pandas as pd


user_data = {}
user_data['tenpo-name'] = 'touhou'
user_data['n-times'] = '3'
user_data['target-date'] = '2023-05-17'

print(user_data['tenpo-name'],user_data['n-times'],user_data['target-date'])
target_day_list = []
number = 0
target_number:int = str(user_data['target-date'][-1])
for i in range(int(user_data['n-times'])):
    while True:
        print(str(datetime.today() - timedelta(days=number))[-1])
        if target_number == str(date.today() - timedelta(days=number))[-1]:
            target_day = date.today() - timedelta(days=number)
            print('取得日',target_day)
            target_day_str = target_day.strftime('%Y-%m-%d')
            target_day_list.append(target_day_str)
            number += 1
            break
        else: 
            pass
        number += 1
target_day_list   

touhou 3 2023-05-17
5
5
取得日 2023-05-17
5
5
5
5
5
5
5
5
5
5
取得日 2023-05-07
5
5
5
5
5
5
5
5
5
5
取得日 2023-04-27


['2023-05-17', '2023-05-07', '2023-04-27']

In [8]:
touhou_df = df[df['店舗名'] == 'マルハン新宿東宝ビル店']
touhou_df[['日付', 'Nの日', '都道府県', '店舗名', '機種名', '台番号', '末尾', 'G数', '差枚',
       'BB', 'RB', 'BB確率', 'RB確率', '合成確率']]

,日付,Nの日,都道府県,店舗名,機種名,台番号,末尾,G数,差枚,BB,RB,BB確率,RB確率,合成確率
93054,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,917,7,2992.0,-800,8.0,7.0,1/374.0,1/427.4,1/199.5
93055,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,918,8,6365.0,100,23.0,21.0,1/276.7,1/303.1,1/144.7
93056,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,919,9,8552.0,-1300,25.0,30.0,1/342.1,1/285.1,1/155.5
93057,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,920,0,5093.0,1800,26.0,13.0,1/195.9,1/391.8,1/130.6
93058,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,921,1,6930.0,1600,31.0,23.0,1/223.5,1/301.3,1/128.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93541,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ゼーガペイン2,995,5,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93542,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,閃乱カグラ BURST UP,996,6,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93543,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ハイスクールD×D2 ハーレム王に俺はなる,997,7,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93544,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ドリフターズ,998,8,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
